<a href="https://colab.research.google.com/github/absolutemocha/sdaai/blob/main/C2329C_AY2022_T3_CWF_Lim_Zhao_Hong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# C2329C Machine Learning Fundamentals

## Coursework Final
***
### Student Name: $<Lim Zhao Hong>$ 
### Student ID: $<20065320>$

***

## Features description from the $CleandDataV20210515.csv$ file:

**index_col** time step for the washing cycle

**avC:** average current 

**avP:** average power

**avR:** average resistant

**maxC:** maximum current 

**maxP:** maximum power

**sdC:** standard deviation for current

**sdP:** standard deviation for power

**stdCR:** standard deviation for resistant

**stdCP:** standard deviation for power

**AvRR:** average relative resistance to previous resistance reading

**mode:** positive class: Daily Wash | negative class: Not Daily Wash


In [ ]:
# Import necessary Python libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Data Preparation Process

In [ ]:
from __future__ import print_function
import os
data_path = ['data']

In [ ]:
# Import the data using the file path
filepath = os.sep.join(data_path + ['CleandDataV20210515.csv'])
data = pd.read_csv(filepath)

In [ ]:
data.head(1).T

In [ ]:
# Remove extraneous columns
data.drop(['state', 'area_code', 'phone_number'], axis=1, inplace=True)

In [ ]:
data.columns


***

## K-Nearest Neigbors

In [ ]:
#K-nearest neighbors requires scaled data. Scale the data using one of the scaling methods discussed in the lecture.


***

## Logistic Regression 

In [ ]:
# Build and evaluate your LR model here ... 



***